In [42]:
%load_ext lab_black

In [43]:
import requests
import json
import datetime
import pandas as pd
from pathlib import Path

In [5]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Funcion para llamar a la API

In [44]:
# Define a function that will take a relative path as an input, call the API, and return a dataframe
def callAPI(relative_path: str, topic_list: bool = False) -> pd.DataFrame:
    base_url = "https://population.un.org/dataportalapi/api/v1"
    target = base_url + relative_path
    # Query string parameters may be appended here or directly in the provided relative path
    # Calls the API
    response = requests.get(target)
    # Reformats response into a JSON object
    j = response.json()
    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
        # If results are paginated, they are transformed into a python dictionary.
        # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j["data"])
        # As long as the nextPage key of the dictionary contains an address for the next API call, the function will continue to call the API and append the results to the dataframe.
        while j["nextPage"] is not None:
            response = requests.get(j["nextPage"])
            j = response.json()
            df_temp = pd.json_normalize(j["data"])
            df = pd.concat([df, df_temp], ignore_index=True)
    except:
        if topic_list:
            df = pd.json_normalize(j, "indicators")
        else:
            df = pd.DataFrame(j)
    return df

In [52]:
# escoger los indicadores a usar
df_indicadores_UNPD = callAPI("/indicators")

In [65]:
df_indicadores_UNPD_selec = df_indicadores_UNPD[
    [
        "id",
        "name",
        "description",
        "shortName",
        "unitLongLabel",
        "topicName",
        "topicShortName",
    ]
].iloc[[0, 14, 15, 16, 19, 42, 43]]

## Geographical areas or locations With Aggregates

In [46]:
# Define target URL.
base_url = "https://population.un.org/dataportalapi/api/v1/locationsWithAggregates?pageNumber=1"

# Call the API and convert the resquest into JSON object.
response = requests.get(base_url).json()

# Convert JSON object to data frame.
df = pd.json_normalize(response)

# Get the response, which includes the first pages. Only 3.
pages = 3

# Converts call into JSON and concat to the previous data frame.
for page in range(2, pages + 1):
    # Reset the target to the next page
    target = f"https://population.un.org/dataportalapi/api/v1/locationsWithAggregates?pageNumber={page}"

    # Each iteration call the API and convert the resquest into JSON object.
    response = requests.get(target).json()

    # Each iteration convert JSON object to data frame.
    df_temp = pd.json_normalize(response)

    # Each iteration concat the data frames.
    df = pd.concat([df, df_temp], ignore_index=True)

# Display only relevant data. (Drop NaN, )
df_paises = (
    df[
        [
            "Id",
            "Name",
            "Iso2",
            "Iso3",
            "Longitude",
            "Latitude",
            "Region",
            "SubRegion",
            "WorldBankIncomeGroup",
            "UNDevelopmentGroup",
        ]
    ]
    .copy()
    .dropna()
)

df.shape (278, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 227 entries, 0 to 236
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Id                    227 non-null    int64  
 1   Name                  227 non-null    object 
 2   Iso2                  227 non-null    object 
 3   Iso3                  227 non-null    object 
 4   Longitude             227 non-null    float64
 5   Latitude              227 non-null    float64
 6   Region                227 non-null    object 
 7   SubRegion             227 non-null    object 
 8   WorldBankIncomeGroup  227 non-null    object 
 9   UNDevelopmentGroup    227 non-null    object 
dtypes: float64(2), int64(1), object(7)
memory usage: 19.5+ KB


In [63]:
# seleccionar los Id de los paises y juntarlos en un strig
country_codes = [str(code) for code in df_paises["Id"].values]
# Converts country code list into a string to be used in later API call
country_selection_string = ",".join(country_codes)
# seleccionar los anios
start_year = 1990
end_year = 2020

# Crear los datasets

In [39]:
df_family_planing = callAPI("/topics/5/indicators")

,topicId,topicName,topicShortName,indicatorId,indicatorName,indicatorDisplayName,indicatorShortName,indicatorDescription,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,unitScaling,formatString,unitLongLabel,unitShortLabel,variableType,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl
1,5,Family Planning,FP,1,Contraceptive prevalence: Any method (Percent),Any,CPAnyP,Percentage of women of reproductive age (15-49 years) who are currently using any method of contraception,False,False,True,True,31,2,4,100,0.01,#0.0,per cent,%,relative,23,Estimates and Projections of Family Planning Indicators,2022,1970,2030,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>Model-based Estimates and Projections of Family Planning Indicators 2022</i>, custom data acquired via website.",https://www.un.org/development/desa/pd/data/family-planning-indicators
9,5,Family Planning,FP,6,Total demand for family planning (Percent),Total demand,DEMTot,Percentage of women of reproductive age (15-49 years) who are currently using any method of contraception or are having unmet need for family planning,False,False,True,True,31,2,4,100,0.01,#0.0,per cent,%,relative,23,Estimates and Projections of Family Planning Indicators,2022,1970,2030,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>Model-based Estimates and Projections of Family Planning Indicators 2022</i>, custom data acquired via website.",https://www.un.org/development/desa/pd/data/family-planning-indicators
11,5,Family Planning,FP,4,Unmet need for family planning: Any method (Percent),Unmet need,UNMP,Percentage of women of reproductive age (15-49 years) who want to stop or delay childbearing but are not using a method of contraception,False,False,True,True,31,2,4,100,0.01,#0.0,per cent,%,relative,23,Estimates and Projections of Family Planning Indicators,2022,1970,2030,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>Model-based Estimates and Projections of Family Planning Indicators 2022</i>, custom data acquired via website.",https://www.un.org/development/desa/pd/data/family-planning-indicators


In [40]:
# Calls the API to return the indicator values for the selected indicators and countries.
# se demoro una hora en cargar la data
df_family_planing_1 = callAPI(
    f"/data/indicators/{1}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)
# Finally, filters the returned results to only include median values for All Women, and limits the number of columns retained in the new dataframe.
df_family_planing_1_= df_family_planing_1.loc[
    (df_family_planing_1["variant"] == "Median")
    & (df_family_planing_1["category"] == "All women"),
    ["location", "iso3", "timeLabel", "indicatorId", "indicator", "value"]
]
df_women_using_MC_1_.to_parquet(f"../datasets/df_UNPD_fam_{1}.parquet")

In [274]:
# Calls the API to return the indicator values for the selected indicators and countries.
# se demoro una hora en cargar la data
df_family_planing_4 = callAPI(
    f"/data/indicators/{4}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)

df_family_planing_4_ = df_family_planing_4.loc[
    (df_family_planing_4["variant"] == "Median")
    & (df_family_planing_4["category"] == "All women"),
    ["location", "iso3", "timeLabel", "indicator", "value"],
]

df_family_planing_4_.to_parquet(f"../datasets/df_UNPD_fam_{4}.parquet")

In [213]:
df_marital_status = callAPI("/topics/6/indicators")

In [215]:
df_marital_status_42 = callAPI(
    f"/data/indicators/{42}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)

### ----
df_marital_status_42_ = df_marital_status_42.loc[
    (df_marital_status_42["variant"] == "Median"),
    ["location", "iso3", "timeLabel", "indicatorId", "indicator", "value"],
]
df_marital_status_42_.to_parquet(f"../datasets/df_UNPD_marstat_{42}.parquet")

In [251]:
df_marital_status_42_.info()

In [220]:
df_child_mortality = callAPI("/topics/8/indicators")
df_child_mortality

,topicId,topicName,topicShortName,indicatorId,indicatorName,indicatorDisplayName,indicatorShortName,indicatorDescription,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,unitScaling,formatString,unitLongLabel,unitShortLabel,variableType,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl
0,8,Child Mortality,IGME,22,Infant mortality rate (IMR),Infant mortality rate (IMR),IMR,Probability of dying between birth and exactly one year of age (1q0),False,True,True,False,188,3,4,0,0.001,##0.0,"per 1,000 live births",per live births,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
1,8,Child Mortality,IGME,24,Under-five mortality rate (U5MR),Under-five mortality rate (U5MR),U5MR,Probability of dying between birth and exactly five years of age (5q0),False,True,True,False,188,3,4,0,0.001,##0.0,"per 1,000 live births",per live births,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/


In [221]:
# Calls the API to return the indicator values for the selected indicators and countries.
df_child_mortality_22 = callAPI(
    f"/data/indicators/{'22'}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)
df_child_mortality_22_ = df_child_mortality_22[
    [
        "location",
        "iso3",
        "timeLabel",
        "indicatorId",
        "indicator",
        "sexId",
        "sex",
        "value",
    ]
]
df_child_mortality_22_.to_parquet(f"../datasets/df_UNPD_mort_{22}.parquet")

In [252]:
df_child_mortality_22_.head(5)

,location,iso3,timeLabel,indicatorId,indicator,sexId,sex,value
0,Afghanistan,AFG,1990,22,Infant mortality rate (IMR),1,Male,134.21248
1,Afghanistan,AFG,1990,22,Infant mortality rate (IMR),2,Female,119.60013
2,Afghanistan,AFG,1990,22,Infant mortality rate (IMR),3,Both sexes,126.95578
3,Afghanistan,AFG,1991,22,Infant mortality rate (IMR),1,Male,130.38545
4,Afghanistan,AFG,1991,22,Infant mortality rate (IMR),2,Female,116.37090


In [225]:
df_child_mortality_24 = callAPI(
    f"/data/indicators/{'24'}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)

df_child_mortality_24_ = df_child_mortality_24[
    [
        "location",
        "iso3",
        "timeLabel",
        "indicatorId",
        "indicator",
        "sexId",
        "sex",
        "value",
    ]
]
df_child_mortality_24_.to_parquet(f"../datasets/df_UNPD_mort_{24}.parquet")

In [254]:
df_child_mortality_24_.head(5)

,location,iso3,timeLabel,indicatorId,indicator,sexId,sex,value
0,Afghanistan,AFG,1990,24,Under-five mortality rate (U5MR),1,Male,198.409690
1,Afghanistan,AFG,1990,24,Under-five mortality rate (U5MR),2,Female,189.829974
2,Afghanistan,AFG,1990,24,Under-five mortality rate (U5MR),3,Both sexes,194.103407
3,Afghanistan,AFG,1991,24,Under-five mortality rate (U5MR),1,Male,191.957315
4,Afghanistan,AFG,1991,24,Under-five mortality rate (U5MR),2,Female,183.570334


In [242]:
df_fertilidad = callAPI("/topics/2/indicators")

In [241]:
df_fertilidad

,topicId,topicName,topicShortName,indicatorId,indicatorName,indicatorDisplayName,indicatorShortName,indicatorDescription,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,unitScaling,formatString,unitLongLabel,unitShortLabel,variableType,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl
0,2,Fertility,Fert,74,Births by age of mother (1-year),Live births by age of mother (and sex of child) - complete,Births1,Number of live births over a given period classified by age group of mother. - complete,True,False,True,False,86,3,4,0,1,### ### ##0,live births,live births,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
1,2,Fertility,Fert,73,Births by age of mother (5-year),Live births by age of mother (and sex of child),Births5,Number of live births over a given period classified by age group of mother.,True,False,True,False,67,3,4,0,1,### ### ##0,live births,live births,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
2,2,Fertility,Fert,55,Crude birth rate,"Crude birth rate (births per 1,000 population)",CBR,Number of live births over a given period divided by the person-years lived by the population over that period.,False,False,True,False,188,3,4,0,1000,#0.0,"births per 1,000 population","per 1,000 population",relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
3,2,Fertility,Fert,68,Fertility rates by age of mother (1-year),Fertility rates by age of mother (1-year),ASFR1,"Number of live births to women at a particular age, divided by the number of women (i.e., person-years) at that age over a given period.",True,False,True,False,86,3,4,0,1,#0.0,per thousand,thousands,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
4,2,Fertility,Fert,17,Fertility rates by age of mother (5-year),Fertility rates by age of mother (5-year),ASFR5,Age-specific fertility rates (ASFR),True,False,True,False,67,3,4,0,1,#0.0,per thousand,thousands,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
5,2,Fertility,Fert,18,Mean age of childbearing (5-year),Mean age of childbearing (5-year),MAC5,Mean age of childbearing,False,False,True,False,188,3,4,0,1,#0.0,years,years,duration,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
6,2,Fertility,Fert,56,Net Reproduction Rate,Net reproduction rate (daughters per woman),NRR,The average number of daughters a hypothetical cohort of women would have at the end of their reproductive period if they were subject during their whole lives to the fertility rates and the mortality rates of a given period. It is expressed as number of daughters per woman.,False,False,True,False,188,3,4,0,1,#0.00,number of daughters per woman,per woman,relative,25,World Population Prospects,2022,1950,2100,"United Nations, Departm

In [232]:
# Calls the API to return the indicator values for the selected indicators and countries.
df_fert_rate = callAPI(
    f"/data/indicators/{19}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)

df_fert_rate_ = df_fert_rate.loc[
    (df_fert_rate["variant"] == "Median"),
    ["location", "iso3", "timeLabel", "indicatorId", "indicator", "value"],
]
df_fert_rate_.to_parquet(f"../datasets/df_UNPD_fert_{19}.parquet")

In [262]:
df_fert_rate_.head(5)

,location,iso3,timeLabel,indicatorId,indicator,value
2,Afghanistan,AFG,1990,19,Total fertility rate,7.564570
7,Afghanistan,AFG,1991,19,Total fertility rate,7.605590
12,Afghanistan,AFG,1992,19,Total fertility rate,7.664820
17,Afghanistan,AFG,1993,19,Total fertility rate,7.718583
22,Afghanistan,AFG,1994,19,Total fertility rate,7.716534


In [245]:
df_imigration = callAPI("/topics/4/indicators")

In [246]:
df_imigration

,topicId,topicName,topicShortName,indicatorId,indicatorName,indicatorDisplayName,indicatorShortName,indicatorDescription,dimAge,dimSex,dimVariant,dimCategory,defaultAgeId,defaultSexId,defaultVariantId,defaultCategoryId,unitScaling,formatString,unitLongLabel,unitShortLabel,variableType,sourceId,sourceName,sourceYear,sourceStartYear,sourceEndYear,sourceCitation,sourceUrl
0,4,International Migration,iMigration,66,Crude rate of net migration,Crude rate of net migration,TNetMigRT,The crude rate of net migration is defined as the ratio of net migration during the year to the average population in that year. The value is expressed per 1000 persons.,False,False,True,False,188,3,4,0,1,### ### ##0,persons,persons,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/
1,4,International Migration,iMigration,65,Total net-migration,Total net number of migrants,TNetMigration,"Net number of migrants, that is, the number of immigrants minus the number of emigrants.",False,False,True,False,188,3,4,0,1,### ### ##0,persons,persons,absolute,25,World Population Prospects,2022,1950,2100,"United Nations, Department of Economic and Social Affairs, Population Division (2022). <i>World Population Prospects: The 2022 Revision</i>, custom data acquired via website.",https://population.un.org/wpp/


In [247]:
# Calls the API to return the indicator values for the selected indicators and countries.
df_migr_rate = callAPI(
    f"/data/indicators/{'66'}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)

df_migr_rate_ = df_migr_rate[
    ["location", "iso3", "timeLabel", "indicatorId", "indicator", "sex", "value"]
]
df_migr_rate_.to_parquet(f"../datasets/df_UNPD_imigrt_{66}.parquet")

In [248]:
df_migr_rate_.head(5)

,locationId,location,iso3,iso2,locationTypeId,indicatorId,indicator,indicatorDisplayName,sourceId,source,revision,variantId,variant,variantShortName,variantLabel,timeId,timeLabel,timeMid,categoryId,category,estimateTypeId,estimateType,estimateMethodId,estimateMethod,sexId,sex,ageId,ageLabel,ageStart,ageEnd,ageMid,value
0,4,Afghanistan,AFG,AF,4,66,Crude rate of net migration,Crude rate of net migration,25,World Population Prospects,0,4,Median,Median,Median,41,1990,1990.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,-60.375
1,4,Afghanistan,AFG,AF,4,66,Crude rate of net migration,Crude rate of net migration,25,World Population Prospects,0,4,Median,Median,Median,42,1991,1991.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,3.721
2,4,Afghanistan,AFG,AF,4,66,Crude rate of net migration,Crude rate of net migration,25,World Population Prospects,0,4,Median,Median,Median,43,1992,1992.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,164.629
3,4,Afghanistan,AFG,AF,4,66,Crude rate of net migration,Crude rate of net migration,25,World Population Prospects,0,4,Median,Median,Median,44,1993,1993.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,87.304
4,4,Afghanistan,AFG,AF,4,66,Crude rate of net migration,Crude rate of net migration,25,World Population Prospects,0,4,Median,Median,Median,45,1994,1994.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,43.986


In [269]:
# Calls the API to return the indicator values for the selected indicators and countries.
df_migr_number = callAPI(
    f"/data/indicators/{'65'}/locations/{country_selection_string}/start/{start_year}/end/{end_year}"
)
df_migr_number_ = df_migr_number[
    ["location", "iso3", "timeLabel", "indicatorId", "indicator", "sex", "value"]
]
df_migr_number_.to_parquet(f"../datasets/df_UNPD_imigrt_{65}.parquet")

In [270]:
df_migr_number.head(5)

,locationId,location,iso3,iso2,locationTypeId,indicatorId,indicator,indicatorDisplayName,sourceId,source,revision,variantId,variant,variantShortName,variantLabel,timeId,timeLabel,timeMid,categoryId,category,estimateTypeId,estimateType,estimateMethodId,estimateMethod,sexId,sex,ageId,ageLabel,ageStart,ageEnd,ageMid,value
0,4,Afghanistan,AFG,AF,4,65,Total net-migration,Total net number of migrants,25,World Population Prospects,0,4,Median,Median,Median,41,1990,1990.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,-666001
1,4,Afghanistan,AFG,AF,4,65,Total net-migration,Total net number of migrants,25,World Population Prospects,0,4,Median,Median,Median,42,1991,1991.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,39926
2,4,Afghanistan,AFG,AF,4,65,Total net-migration,Total net number of migrants,25,World Population Prospects,0,4,Median,Median,Median,43,1992,1992.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,1834556
3,4,Afghanistan,AFG,AF,4,65,Total net-migration,Total net number of migrants,25,World Population Prospects,0,4,Median,Median,Median,44,1993,1993.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,1171818
4,4,Afghanistan,AFG,AF,4,65,Total net-migration,Total net number of migrants,25,World Population Prospects,0,4,Median,Median,Median,45,1994,1994.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,665410


## OMS data

In [20]:
OMS = "https://ghoapi.azureedge.net/api/"
response = requests.get("https://ghoapi.azureedge.net/api/Indicator")
j = response.json()
df_indicadores = pd.json_normalize(j["value"])

In [41]:
#buscar indicadores
indicadores = 'ana'
df_indicadores[df_indicadores['IndicatorName'].str.contains(indicadores, case=False)]

,IndicatorCode,IndicatorName,Language
2027,NUTRITION_ANAEMIA_NONPREGNANT_PREV,Prevalence of anaemia in non-pregnant women (aged 15-49) (%),EN
2051,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV,Prevalence of anaemia in women of reproductive age (aged 15-49) (%),EN
2070,NUTRITION_ANAEMIA_REPRODUCTIVEAGE_NUM,Number of women of reproductive age (aged 15-49 years) with anaemia (thousands),EN
2094,NUTRITION_ANAEMIA_CHILDREN_NUM,Number of children aged 6–59 months with anaemia (thousands),EN
2108,NUTRITION_ANAEMIA_PREGNANT_NUM,Number of pregnant women (aged 15-49 years) with anaemia (thousands),EN
2122,NUTRITION_ANAEMIA_CHILDREN_PREV,Prevalence of anaemia in children aged 6–59 months (%),EN
2207,NUTRITION_ANAEMIA_NONPREGNANT_NUM,Number of non-pregnant women (aged 15-49 years) with anaemia (thousands),EN
2219,NUTRITION_ANAEMIA_PREGNANT_PREV,Prevalence of anaemia in pregnant women (aged 15-49) (%),EN


In [6]:
def callAPI_OMS(IndicatorCode: str):
    OMS = "https://ghoapi.azureedge.net/api/"
    target = OMS + IndicatorCode + "?pageNumber=1"
    response = requests.get(target)
    j = response.json()
    df = pd.json_normalize(j["value"])
    pages = 100
    # Converts call into JSON and concat to the previous data frame.
    for page in range(2, pages + 1):
        # Reset the target to the next page
        target = OMS + IndicatorCode + f"?pageNumber={page}"
            
        # Each iteration call the API and convert the resquest into JSON object.
        response = requests.get(target).json()

        # Each iteration convert JSON object to data frame.
        df_temp = pd.json_normalize(response)

        # Each iteration concat the data frames.
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

## Decargar los datasets

In [70]:
indicadores_OMS = [
    "NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV",
    "NCD_BMI_30A",
    "NUTRITION_ANAEMIA_CHILDREN_PREV",
    "NCD_BMI_30A",
    "M_Est_cig_curr",
    "SA_0000001688",
]
for code in indicadores_OMS:
    dataset_OMS = callAPI_OMS(code)
    dataset_OMS = dataset_OMS[
        [
            "Id",
            "IndicatorCode",
            "SpatialDim",
            "TimeDimensionValue",
            "Dim1Type",
            "Dim1",
            "NumericValue",
            "Low",
            "High",
            "Comments",
        ]
    ]
    dataset_OMS.to_parquet(f"../datasets/df_OMS_{code}.parquet")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
#Pocos datos para poder analizarlos
df_obesidad_ninios = callAPI_OMS('NUTRITION_ANT_WHZ_NE2')
df_desnutricion_niños_2 = callAPI_OMS("sevwast5")
df_crecimiento_niños = callAPI_OMS('NUTRITION_ANT_HAZ_NE2')
df_desnutricion_niños = callAPI_OMS('NUTRITION_WH_3')
df_gastos_salud = callAPI_OMS("FINPROTECTION_CATA_TOT_25_POP")

In [77]:
df_gastos_salud.head()

,Id,IndicatorCode,SpatialDimType,SpatialDim,TimeDimType,TimeDim,Dim1Type,Dim1,Dim2Type,Dim2,Dim3Type,Dim3,DataSourceDimType,DataSourceDim,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd,@odata.context,value
0,28062423.0,FINPROTECTION_CATA_TOT_25_POP,COUNTRY,USA,YEAR,2005.0,RESIDENCEAREATYPE,TOTL,None,None,None,None,DATASOURCE,FPUSA2005,0.97,0.97,None,None,Consumption based survey-based estimate,2022-01-31T15:54:11.597+01:00,2005,2005-01-01T00:00:00+01:00,2005-12-31T00:00:00+01:00,NaN,NaN
1,28062424.0,FINPROTECTION_CATA_TOT_25_POP,COUNTRY,USA,YEAR,2005.0,RESIDENCEAREATYPE,URB,None,None,None,None,DATASOURCE,FPUSA2005,0.89,0.89,None,None,Consumption based survey-based estimate,2022-01-31T15:54:11.627+01:00,2005,2005-01-01T00:00:00+01:00,2005-12-31T00:00:00+01:00,NaN,NaN
2,28062425.0,FINPROTECTION_CATA_TOT_25_POP,COUNTRY,USA,YEAR,2005.0,RESIDENCEAREATYPE,RUR,None,None,None,None,DATASOURCE,FPUSA2005,1.79,1.79,None,None,Consumption based survey-based estimate,2022-01-31T15:54:11.643+01:00,2005,2005-01-01T00:00:00+01:00,2005-12-31T00:00:00+01:00,NaN,NaN
3,28067734.0,FINPROTECTION_CATA_TOT_25_POP,COUNTRY,HUN,YEAR,2004.0,RESIDENCEAREATYPE,TOTL,None,None,None,None,DATASOURCE,FPHUN2004,0.90,0.90,None,None,Consumption based survey-based estimate,2022-01-31T15:56:05.197+01:00,2004,2004-01-01T00:00:00+01:00,2004-12-31T00:00:00+01:00,NaN,NaN
4,28067735.0,FINPROTECTION_CATA_TOT_25_POP,COUNTRY,HUN,YEAR,2004.0,RESIDENCEAREATYPE,URB,None,None,None,None,DATASOURCE,FPHUN2004,0.90,0.90,None,None,Consumption based survey-based estimate,2022-01-31T15:56:05.21+01:00,2004,2004-01-01T00:00:00+01:00,2004-12-31T00:00:00+01:00,NaN,NaN
